In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
root_path = 'gdrive/My Drive/iiitd_research'

In [ ]:
import os
print(os.listdir())
os.chdir('gdrive/My Drive/iiitd_research')
#!unzip 'constraint22_dataset_covid19.zip'
# print(os.getcwd())
# print(os.listdir('./content/covid_meme_data'))


#os.chdir('/content/view?usp=sharing')

['.config', 'gdrive', 'sample_data']


In [ ]:
import json
with open('constraint22_dataset_unseen_test_gold_labels.jsonl') as f:
    testdata = [json.loads(line) for line in f]
print(testdata[4])

{'OCR': 'TOGETHER THEY RAPED A 13 YR OLD GIRL\nTHEY ARE BACKIN COURT DECEMBER 16/2016\n', 'image': 'memes_1731.png', 'hero': [], 'villain': ['jeffrey epstein', 'donald trump'], 'victim': ['13 year old girl'], 'other': []}


In [ ]:
test = []
k=0
for i in testdata :
  i['entities']=i['hero']+i['villain']+i['victim']+i['other']

  if i['image'].startswith('m'):
    for en in i['entities']:
      new_dict={}

      new_dict['image'] = i['image']
      new_dict['OCR'] = i['OCR']
      new_dict['entity'] = en
      if en in i['hero']:
        new_dict['label'] =0 #'hero'
      elif en in i['villain'] :
        new_dict['label'] =1 #'villain'
      elif en in i['victim'] :
        new_dict['label'] =2 #'victim'
      else:
        new_dict['label'] =3 #'other'


      test.append(new_dict)
  k=k+1

In [ ]:
print(test[5])

{'image': 'memes_3330.png', 'OCR': 'APPARENTLY\nCAKES\nARE MORE IMPORTANT\nTOYOU THANALLOFTHIS:\nTAIX\nSTOPPING BAANE STRIKIS INUING THE FOICE STATE ELMINATING TLES\nBRING\nTHE\nTROOPS\nHOME\nSudget\nNOW\nLEGALIZE\nINDING THE\nBINGING OR\nTROOPS OME\nALANCIN\nTHE UDG\nLERT RIGHTS\nEND\nTHESE\nWARS\nDIT TE D\nSIPAnON Or\nCHURCHA STATE\nNO MORE\nCON CONTROL\nWARS\n', 'entity': 'people', 'label': 3}


In [ ]:
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch


In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

     |████████████████████████████████| 53 kB 1.3 MB/s 
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ixwjvvkt
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-ixwjvvkt
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369387 sha256=f835b51dad66d09afc285966705e9ee9b2a013030d9088a03d923d32c63784e7
  Stored in directory: /tmp/pip-ephem-wheel-cache-sl6fqcbi/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [ ]:
import clip

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [ ]:
model, preprocess = clip.load("RN50x16")
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

100%|███████████████████████████████████████| 630M/630M [00:17<00:00, 37.2MiB/s]


Model parameters: 290,979,217
Input resolution: 384
Context length: 77
Vocab size: 49408


In [ ]:
testimages = []
testocr = []
testentity=[]
testlabels = []


temp=''
for i in test:
  if i['image'] not in os.listdir('test_data/images'):
        continue
  testocr.append(i['OCR'])
  testentity.append(i['entity'])
  testlabels.append(i['label'])



  # if i['image']!=temp:
  #     image = Image.open(os.path.join('test_data/images/', i['image'])).convert("RGB")
  #     testimages.append(preprocess(image))
  #     temp=i['image']
  # else:
  #     testimages.append(testimages[-1])







In [ ]:
len(testocr)

1143

In [ ]:
len(testimages)

0

In [ ]:
#(testentity.shape)

In [ ]:
k=0
testimage_features = []#torch.Tensor(3000,640)
for i in range(129):

  testimage_input = torch.tensor(np.stack(testimages[k:k+10])).cuda()
  k+=10
  with torch.no_grad():
    testimage_features=testimage_features + model.encode_image(testimage_input).float().tolist()

In [ ]:
#testimage_input = torch.tensor(np.stack(testimages)).cuda()
testocr_tokens = clip.tokenize([ o for o in testocr],truncate=True).cuda()
testentity_tokens = clip.tokenize([ e for e in testentity]).cuda()

In [ ]:

with torch.no_grad():
    #testimage_features= model.encode_image(testimage_input).float()
    testocr_features = model.encode_text(testocr_tokens).float()
    testentity_features = model.encode_text(testentity_tokens).float()

In [ ]:
#print(testimage_features)
print(testocr_features.shape)
print(testentity_features.shape)

torch.Size([1290, 768])
torch.Size([1290, 768])


In [ ]:
testimage_features=torch.Tensor(testimage_features).cuda()

In [ ]:
test_tensor = torch.cat((testimage_features, testocr_features, testentity_features), 1)

In [ ]:
import pickle
with open("RN50x16testembeddings.pickle", 'wb') as f:
    pickle.dump(test_tensor, f)



In [ ]:
import pickle
with open("testlabels.pickle", 'wb') as f:
    pickle.dump(testlabels, f)



bert test embeddings

In [ ]:
!python -m pip install transformers

from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import torch

bertmodel = BertModel.from_pretrained('bert-base-uncased',
           output_hidden_states = True,)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.7 MB/s 
     |████████████████████████████████| 6.6 MB 50.7 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 66.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
def bert_text_preparation(text, tokenizer):
    """Preparing the input for BERT

    Takes a string argument and performs
    pre-processing like adding special tokens,
    tokenization, tokens to ids, and tokens to
    segment ids. All tokens are mapped to seg-
    ment id = 1.

    Args:
        text (str): Text to be converted
        tokenizer (obj): Tokenizer object
            to convert text into BERT-re-
            adable tokens and ids

    Returns:
        list: List of BERT-readable tokens
        obj: Torch tensor with token ids
        obj: Torch tensor segment ids


    """

    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer(marked_text, padding='max_length', max_length = 100, truncation=True,return_tensors="pt")
    # indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # segments_ids = [1]*len(indexed_tokens)

    # Convert inputs to PyTorch tensors
    # tokens_tensor = torch.tensor([indexed_tokens])
    # segments_tensors = torch.tensor([segments_ids])

    return tokenized_text#, tokens_tensor, segments_tensors

In [ ]:
def get_bert_embeddings(tokenized_text,model):#tokens_tensor, segments_tensors, model):
    """Get embeddings from an embedding model

    Args:
        tokens_tensor (obj): Torch tensor size [n_tokens]
            with token ids for each token in text
        segments_tensors (obj): Torch tensor size [n_tokens]
            with segment ids for each token in text
        model (obj): Embedding model to generate embeddings
            from token and segment ids

    Returns:
        list: List of list of floats of size
            [n_tokens, n_embedding_dimensions]
            containing embeddings for each token

    """
    use_cuda = torch.cuda.is_available()
    # device = torch.device("cuda" if use_cuda else "cpu")
    device = torch.device( "cpu")
    mask = tokenized_text['attention_mask'].to(device)
    input_id = tokenized_text['input_ids'].squeeze(1).to(device)
    # Gradient calculation id disabled
    # Model is in inference mode
    with torch.no_grad():
        #outputs = model(tokens_tensor, segments_tensors)
        outputs = model(input_id, mask)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]

    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

In [ ]:
bert_test_embeddings = []
p=0
for text in testocr:
    tokenized_text = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokenized_text, bertmodel)
    p+=1
    if p%50==0:
      print('still running, at ',p)
    # Find the position 'bank' in list of tokens
    #word_index = tokenized_text.index('bank')
    # Get the embedding for bank
    #word_embedding = list_token_embeddings[word_index]
    # word=[]
    # np.sum(list_token_embeddings,axis=0)/100
    # for e in list_token_embeddings:
    #   word+=  e/len(list_token_embeddings)

    bert_test_embeddings.append(np.sum(list_token_embeddings,axis=0)/100)


still running, at  50
still running, at  100
still running, at  150
still running, at  200
still running, at  250
still running, at  300
still running, at  350
still running, at  400
still running, at  450
still running, at  500
still running, at  550
still running, at  600
still running, at  650
still running, at  700
still running, at  750
still running, at  800
still running, at  850
still running, at  900
still running, at  950
still running, at  1000
still running, at  1050
still running, at  1100


In [ ]:
(bert_test_embeddings[3])

In [ ]:
import pickle
with open("uspoliticsberttestembeddings.pickle", 'wb') as f:
    pickle.dump(bert_test_embeddings, f)


In [ ]:

bert_test_entity_embeddings = []
p=0
for text in testentity:
    tokenized_text = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokenized_text, bertmodel)
    p+=1
    if p%50==0:
      print('still running, at ',p)
    # Find the position 'bank' in list of tokens
    #word_index = tokenized_text.index('bank')
    # Get the embedding for bank
    #word_embedding = list_token_embeddings[word_index]
    # word=[]
    # np.sum(list_token_embeddings,axis=0)/100
    # for e in list_token_embeddings:
    #   word+=  e/len(list_token_embeddings)

    bert_test_entity_embeddings.append(np.sum(list_token_embeddings,axis=0)/100)


still running, at  50
still running, at  100
still running, at  150
still running, at  200
still running, at  250
still running, at  300
still running, at  350
still running, at  400
still running, at  450
still running, at  500
still running, at  550
still running, at  600
still running, at  650
still running, at  700
still running, at  750
still running, at  800
still running, at  850
still running, at  900
still running, at  950
still running, at  1000
still running, at  1050
still running, at  1100


In [ ]:
len(bert_test_entity_embeddings[3])

768

In [ ]:
import pickle
with open("uspoliticsberttestentityembeddings.pickle", 'wb') as f:
    pickle.dump(bert_test_entity_embeddings, f)
